In [3]:
#install needed module
%pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

## How To Set Up Authentication


1.   Make sure you have a Spotify for Developers Account: https://developer.spotify.com/

2. Create an App with Redirect URL "http://localhost:8888/callback"

3. Add yourself as a user in the App Settings with your spotify username and email

4. Set the username variable below to your Spotify username

5. Get the app client id and put it into the client_id variable below

6. Get the app secret id and put it into the client_secret variable below

7. Run the cell




In [59]:
# Set up authentication with the Spotify API
username = 'enter username'
scope = 'user-library-read'
client_id = 'enter client id'
client_secret = 'enter client secret id'
redirect_uri = 'http://localhost:8888/callback'

In [39]:
from spotipy.oauth2 import SpotifyOAuth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, username=username, open_browser=False))

The following block of code will tell you to open a URL. Once you open it, copy the URL you were redirected to and paste it in the prompt box that will appear underneath the code block. Even if the redirection tells you there is an error, this will work.

In [40]:
# Get the user's top tracks
#top_tracks = sp.current_user_top_tracks(limit=50, time_range='short_term')
top_tracks = sp.current_user_saved_tracks()

# Create a dataframe of track features
features = ['id', 'name', 'artists', 'popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
track_df = pd.DataFrame(columns=features)


In [41]:
#prompt user to enter mood
mood = input("Enter a mood out of the following: Happy, Sad, Energetic, Calm: ")

Enter a mood out of the following: Happy, Sad, Energetic, Calm: Sad


In [42]:
moods = ['Happy', 'Sad', 'Energetic', 'Calm', 'happy', 'sad', 'energetic', 'calm']
if mood not in moods:
  print("Error: Selected mood not in available options. Please re-input the mood and run this cell again")

In [43]:
#print(top_tracks['items'][0]['track'])
track = top_tracks['items'][0]['track']
track_id = track['name']
print(track_id)

Marry You


In [44]:
for i in range(len(top_tracks['items'])):
    track = top_tracks['items'][i]['track']
    track_id = track['id']
    track_name = track['name']
    track_artists = ', '.join([artist['name'] for artist in track['artists']])
    track_popularity = track['popularity']
    track_features = sp.audio_features(track_id)[0]
    track_df.loc[i] = [track_id, track_name, track_artists, track_popularity, track_features['danceability'], track_features['energy'], track_features['loudness'], track_features['speechiness'], track_features['acousticness'], track_features['instrumentalness'], track_features['liveness'], track_features['valence'], track_features['tempo']]

In [45]:
track_df.head()

,id,name,artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,22PMfvdz35fFKYnJyMn077,Marry You,Bruno Mars,76,0.621,0.820,-4.865,0.0367,0.3320,0.000000,0.104,0.4520,144.905
1,2tJulUYLDKOg9XrtVkMgcJ,Grenade,Bruno Mars,79,0.704,0.558,-7.273,0.0542,0.1480,0.000000,0.107,0.2450,110.444
2,161DnLWsx1i3u1JT05lzqU,Talking to the Moon,Bruno Mars,82,0.498,0.590,-4.721,0.0320,0.5110,0.000000,0.107,0.0784,145.867
3,3w3y8KPTfNeOKPiqUTakBh,Locked out of Heaven,Bruno Mars,88,0.726,0.698,-4.165,0.0431,0.0490,0.000000,0.309,0.8670,143.994
4,06PpOnH3uJuwu8cin6sTlw,Halogen Nights,Denis Coleman,19,0.698,0.879,-4.068,0.0828,0.0784,0.000002,0.115,0.7980,146.907


In [46]:
# Normalize the features
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(track_df[['popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])

In [47]:
# Compute cosine similarity between tracks
cosine_sim = cosine_similarity(normalized_features)

In [48]:
# Create a neural network model
model = Sequential()
model.add(Dense(64, input_shape=(len(features)-3,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [49]:
# Train the model
X_train = normalized_features[:]
y_train = normalized_features[:,1]
model.fit(X_train, y_train, epochs=10, batch_size=50)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.1909
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 0.2014
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.1744
Epoch 4/10
1/1 [==============================] - 0s 14ms/step - loss: 0.1745
Epoch 5/10
1/1 [==============================] - 0s 12ms/step - loss: 0.1636
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 0.0623
Epoch 7/10
1/1 [==============================] - 0s 10ms/step - loss: 0.1805
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 0.1122
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.1123
Epoch 10/10
1/1 [==============================] - 0s 10ms/step - loss: 0.2557


In [50]:
df_mood = pd.read_csv('./data_moods.csv')
df_mood.head()

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,0.866,0.13700,0.730,0.000000,0.0843,0.625,-8.201,0.0767,118.523,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,0.381,0.01890,0.832,0.196000,0.1530,0.166,-5.069,0.0492,120.255,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,0.346,0.91300,0.139,0.000077,0.0934,0.116,-15.326,0.0321,136.168,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,0.466,0.08900,0.438,0.000006,0.1130,0.587,-12.858,0.0608,193.100,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,0.419,0.00171,0.932,0.000000,0.1370,0.445,-3.604,0.1060,169.881,1,4,Energetic


In [51]:
normalized_features2 = scaler.fit_transform(df_mood[['popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])

In [52]:
X_test = normalized_features2[:]

In [53]:
# Compute cosine similarity between tracks
cosine_sim2 = cosine_similarity(normalized_features2)

In [54]:
# Predict popularity of tracks
predicted_popularity = model.predict(X_test)

# Combine the predicted popularity and cosine similarity into a recommendation score
recommendation_scores = predicted_popularity.flatten() * cosine_sim2[0]

# Get the top 10 recommended tracks
recommended_track_indices = recommendation_scores.argsort()[-10:][::-1]

# Get the top recommended track
recommended_track = recommendation_scores.argsort()[-1:][::-1]
print('Top track: ', df_mood.iloc[recommended_track]['name'])

# Print the recommended tracks without sentiment analysis
print('Recommended tracks:')
for i in recommended_track_indices:
    print(df_mood.iloc[i]['name'] + ' - ' + df_mood.iloc[i]['artist'] + ' - ' + str(df_mood.iloc[i]['popularity']))


22/22 [==============================] - 0s 2ms/step
Top track:  25    All or Nothing (feat. Axel Ehnström) - Deluxe Mix
Name: name, dtype: object
Recommended tracks:
All or Nothing (feat. Axel Ehnström) - Deluxe Mix - Lost Frequencies - 22
comethru - Jeremy Zucker - 78
Mr. Blue Sky - Electric Light Orchestra - 82
Dolerme - ROSALÍA - 68
Into Black - Blouse - 47
Tilted Towers - Alesso - 50
Anthem - EDX - 44
Emotions in Motion - Jaakko Eino Kalevi - 23
Don’t Delete The Kisses - Wolf Alice - 59
Somebody That I Used To Know - Gotye - 77


In [56]:
#Split dataframe between songs that match the mood and those that don't 
dfRecommendedSongs = df_mood.iloc[recommended_track_indices]

dfMoodSongs = dfRecommendedSongs[dfRecommendedSongs['mood'].str.lower() == mood.lower()]
dfNotMoodSongs = dfRecommendedSongs[dfRecommendedSongs['mood'].str.lower() != mood.lower()]

dfFinalRecommendation = pd.concat([dfMoodSongs, dfNotMoodSongs])

In [57]:
dfFinalRecommendation.head(10)

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
671,comethru,"summer,",Jeremy Zucker,18uwL0vNUanqZH0ro2QcOP,2018-09-28,78,181613,0.795,0.607000,0.178,0.000000,0.6230,0.608,-11.991,0.0574,93.982,0,4,Sad
144,Dolerme,Dolerme,ROSALÍA,2reL9PAUza4PUtbjqW1fRf,2020-03-24,68,143975,0.545,0.785000,0.478,0.002780,0.1230,0.318,-7.035,0.0559,173.746,8,4,Sad
272,Into Black,Blouse,Blouse,5d8pKJxoDlYX2SoFR6RCsm,2011-10-31,47,208237,0.705,0.463000,0.681,0.839000,0.1230,0.707,-8.278,0.0258,114.003,9,4,Sad
165,Emotions in Motion,Emotions in Motion,Jaakko Eino Kalevi,4UmpEQtTlhBOxDPePaK8qR,2018-08-01,23,256906,0.651,0.199000,0.503,0.644000,0.0781,0.558,-10.906,0.0323,133.044,0,4,Sad
148,Don’t Delete The Kisses,Visions Of A Life,Wolf Alice,1LomCMl5wD3MPVlC6smUHe,2017-09-29,59,275226,0.604,0.001100,0.789,0.828000,0.3680,0.348,-6.259,0.0273,122.032,5,4,Sad
25,All or Nothing (feat. Axel Ehnström) - Deluxe Mix,Less is More (Deluxe),Lost Frequencies,15bAMg00o4VdWmQrlkbfG3,2017-07-14,22,170458,0.737,0.226000,0.816,0.453000,0.6210,0.533,-9.220,0.0411,124.014,4,4,Happy
356,Mr. Blue Sky,Out of the Blue,Electric Light Orchestra,2RlgNHKcydI9sayD2Df2xp,1977,82,303373,0.388,0.652000,0.338,0.000004,0.2480,0.477,-10.054,0.0328,177.784,10,4,Happy
573,Tilted Towers,Tilted Towers,Alesso,5hdAQegqnitMjNacfLOHQd,2018-11-30,50,259475,0.702,0.000771,0.842,0.828000,0.2800,0.175,-5.059,0.0331,124.010,7,4,Happy
33,Anthem,Anthem,EDX,1SSAvErVyid5OXzmtGZKkg,2018-03-30,44,158483,0.664,0.001520,0.860,0.894000,0.1970,0.249,-4.505,0.0345,121.991,7,4,Happy
494,Somebody That I Used To Know,Making Mirrors,Gotye,1qDrWA6lyx8cLECdZE7TV7,2011-01-01,77,244884,0.865,0.548000,0.521,0.000115,0.0989,0.748,-6.932,0.0371,129.059,0,4,Happy


In [58]:
# Print the sentiment recommended tracks with sentiment analysis
print('Recommended tracks:')
for i in range(10):
    print(dfFinalRecommendation.iloc[i]['name'] + ' - ' + dfFinalRecommendation.iloc[i]['artist'] + ' - ' + str(df_mood.iloc[i]['popularity']))

Recommended tracks:
comethru - Jeremy Zucker - 68
Dolerme - ROSALÍA - 43
Into Black - Blouse - 60
Emotions in Motion - Jaakko Eino Kalevi - 2
Don’t Delete The Kisses - Wolf Alice - 60
All or Nothing (feat. Axel Ehnström) - Deluxe Mix - Lost Frequencies - 27
Mr. Blue Sky - Electric Light Orchestra - 29
Tilted Towers - Alesso - 1
Anthem - EDX - 68
Somebody That I Used To Know - Gotye - 0


As is evident by the results above, the recommendation list with user sentiment results in a different order of recommendations.